In [13]:
from pywr.core import Model
from pywr.notebook import draw_graph
import matplotlib.pyplot as plt
import os
from pywr.domains.groundwater import KeatingAquifer
from pywr.parameters.groundwater import KeatingStreamFlowParameter
from pywr.core import Model, Input, Output, Link, Storage, Node, Catchment
from pywr.recorders import (
    NumpyArrayNodeRecorder,
    NumpyArrayLevelRecorder,
    NumpyArrayStorageRecorder,
)
import pandas
import numpy as np
import pytest

def createAquifer(model,name):
    num_streams = 1
    num_additional_inputs = 0
    stream_flow_levels = [[1.0, 1.25]]  # m
    transmissivity = [100, 2000]  # m2/d
    transmissivity = [t * 0.001 for t in transmissivity]  # m3 to Ml
    coefficient = 1  # no units
    storativity = [0.0005]  # % este parámetro es clave
    levels = [1.0, 10.0]  # m
    area = 50000 * 50000  # m2
    
    aqfer=KeatingAquifer(
        model,
        name,
        num_streams,
        num_additional_inputs,
        stream_flow_levels,
        transmissivity,
        coefficient,
        levels,
        area=area,
        storativity=storativity)

    aqfer.initial_level=1
    
    return aqfer

if __name__ == "__main__":
    
    path="nuble.json"
    model = Model.load(path)
        
    aqfer=createAquifer(model,'AC01_NIQ')
        
    model.nodes['REMZemita'].connect(aqfer)
#     model.nodes['Zemita_b'].connect(aqfer)
    aqfer.connect(model.nodes['abstraccionAq'],
                            from_slot=0)

    model.solver.name="glpk-edge"
    stats = model.run()
    print(stats)
    
    plt.figure()
    draw_graph(model,labels=True,attributes=True,
               width=900,height=900)
    plt.show()


Model executed 1 scenarios in 0.4 seconds, running at 999.8 timesteps per second.


<IPython.core.display.Javascript object>

<Figure size 640x480 with 0 Axes>

In [14]:
#validar el balance de masa en canal Virguin
print(model.nodes["FRVirguin"].flow)
print(model.nodes["VirguinJFR"].flow)
print(model.nodes["REMVirguinJFR"].flow)
#validar balance de masa en canal Zemita
print(model.nodes["REMVirguinJFR"].flow)


[28.57142857]
[14.28571429]
[14.28571429]
[14.28571429]


In [2]:
# from pywr.notebook import to_html
# to_html(m,height=800,width=800,labels=True,attributes=True,title='Modelo Cuenca del Río Huasco')

In [10]:
from pywr.notebook import pywr_model_to_d3_json
d3json=pywr_model_to_d3_json(model)
import json
with open("./d3.json", "w") as fp:
    json.dump(d3json,fp) 